In [1]:
from roman_pointing.roman_observability import (
    get_target_coords, 
    compute_keepout, 
    plot_keepout,
    plot_solar_angle,
    compute_roman_angles,
    plot_pitch
)
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# UI Widgets
target_input = widgets.Textarea(
    value='Proxima Cen\nSirius\nBetelgeuse',
    placeholder='Enter target names (one per line)',
    description='Targets:',
    layout=widgets.Layout(width='500px', height='100px')
)

start_date_input = widgets.Text(
    value='2027-01-01T00:00:00',
    description='Start Date:',
    layout=widgets.Layout(width='400px')
)

days_input = widgets.IntSlider(
    value=365,
    min=1,
    max=730,
    description='Days:',
    layout=widgets.Layout(width='400px')
)

time_step_input = widgets.IntSlider(
    value=1,
    min=1,
    max=10,
    description='Time Step:',
    layout=widgets.Layout(width='400px')
)

run_button = widgets.Button(
    description='Generate Maps & Plots',
    button_style='primary',
    layout=widgets.Layout(width='200px')
)

output = widgets.Output()

# Event handler
def on_run_clicked(b):
    with output:
        clear_output(wait=True)
        
        try:
            # Parse targets
            target_names = [t.strip() for t in target_input.value.split('\n') if t.strip()]
            
            print(f"Processing {len(target_names)} targets...")
            
            # Get coordinates
            coords = get_target_coords(target_names)
            print(f"Found {len(coords)} targets")
            
            if len(coords) == 0:
                print("Error: No valid targets found!")
                return
            
            # Compute keepout
            ts, keepout, solar_angles = compute_keepout(
                coords,
                start_date_input.value,
                days_input.value,
                time_step_input.value
            )
            
            # Compute pitch angles for all targets
            pitch_dict = {}
            for name, coord in coords.items():
                _, _, _, pitch = compute_roman_angles(
                    coord,
                    start_date_input.value,
                    days_input.value,
                    time_step_input.value
                )
                pitch_dict[name] = pitch
            
            # Plot keepout map
            fig1, ax1 = plot_keepout(keepout, ts)
            plt.show()
            
            # Plot solar angles
            fig2, ax2 = plot_solar_angle(ts, solar_angles)
            plt.show()
            
            # Plot pitch angles
            fig3, ax3 = plot_pitch(ts, pitch_dict)
            plt.show()
            
        except Exception as e:
            print(f"Error: {e}")
            import traceback
            traceback.print_exc()

run_button.on_click(on_run_clicked)

# Display UI
display(widgets.VBox([
    widgets.HTML("<h2>🪐 Roman Space Telescope Keepout Map Generator 🪐</h2>"),
    target_input,
    start_date_input,
    days_input,
    time_step_input,
    run_button,
    output
]))